# Geostatistics

## 4. Spatial Structure

With this lecture we will start to look at *distance* and *spatial* relations of observations. We will cover

* the concept of *distance*
* talk (again) about co-variance
* introduce variograms

*Please be patient, this lecture is still under construction*